# Figure 4: Behaviour of sulfur

In [1]:
# import python packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import VolFe as vf
import math

## Functions to calculate sulfur-related variables

Kress, V.C. and Carmichael, I.S., 1991. The compressibility of silicate liquids containing Fe2O3 and the effect of composition, temperature, oxygen fugacity and pressure on their redox states. Contributions to Mineralogy and Petrology, 108, pp.82-92. https://doi.org/10.1007/BF00307328

Frost, B., 1991. Introduction to oxygen fugacity and its petrologic importance. Reviews in Mineralogy and Geochemistry, 25, pp.1-9. https://doi.org/10.1515/9781501508684-004

O'Neill, H.S.C., 2021. The thermodynamic controls on sulfide saturation in silicate melts with application to ocean floor basalts. Magma redox geochemistry, pp.177-213. https://doi.org/10.1002/9781119473206.ch10

O'Neill, H.S.C. and Mavrogenes, J.A., 2022. The sulfate capacities of silicate melts. Geochimica et Cosmochimica Acta, 334, pp.368-382. https://doi.org/10.1016/j.gca.2022.06.020

Chowdhury, P. and Dasgupta, R., 2019. Effect of sulfate on the basaltic liquidus and sulfur Concentration at Anhydrite Saturation (SCAS) of hydrous basalts–Implications for sulfur cycle in subduction zones. Chemical Geology, 522, pp.162-174. https://doi.org/10.1016/j.chemgeo.2019.05.020

Ohmoto, H., and Kerrick, D., 1977. Devolatilization equilibria in graphitic systems. American Journal of Science, 277(8), pp.1013-1044. https://doi.org/10.2475/AJS.277.8.1013 

VolFe: https://github.com/eryhughes/VolFe

In [2]:
# Choose which parameterisations to use to calculate the various sulfur-related variables
my_models = [['fO2','Kress91A'], # Model for parameterisation of relationship between fO2 and Fe3+/FeT: Eq. (A-5, A-6) in Kress and Carmichael (1991)
                ['FMQbuffer','Frost91'], # Model for the parameterisation for the fO2 value of the FMQ buffer: FM[beta]Q in Table 1 of Frost (1991)
                ['sulfide','ONeill21dil'], # Model for the parameterisation for the *S2- solubility constant: Eq. (10.34) inc. H2O dilution from O'Neill (2021)
                ['sulfate','ONeill22dil'], # Model for the parameterisation of the S6+ solubility constant: Eq. (12a) inc. H2O dilution from O'Neill & Mavrogenes (2022)
                ['SCSS','ONeill21hyd'], # Model for parameterisation of the sulfide content at sulfide saturation (S2-CSS): Eq. (10.34, 10.43, 10.45, 10.46, 10.49) from O'Neill (2021)
                ['SCAS','Chowdhury19'], # Model for parameterisation of the sulfate content at anhydrite saturation (S6+CAS): Eq. (8) using Table 5 in Chowdhury & Dasgupta (2019)
                ['KHOSg','Ohmoto97'], # Model for the parameterisation of the equilibiurm constant for 0.5S2 + H2O = H2S + 0.5O2: Reaction (h) in Table 1 from Ohmoto & Kerrick (1977)
                ['KOSg','Ohmoto97']] # Model for the parameterisation of the equilibiurm constant for 0.5S2 + O2 = SO2: Reaction (f) in Table 1 from Ohmoto & Kerrick (1977)

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# calculate sulfur-related variables with varying melt composition
def calc_sulfur_vcomp(setup,models=vf.default_models):
    for n in range(0,len(setup),1): # loop over rows in setup file
        melt_wf=vf.melt_comp(n,setup) # set the melt composition
        PT = {"T":setup.loc[n,"T_C"],"P":setup.loc[n,"P_bar"]} # set the P and T
        melt_wf['CO2'] = setup.loc[n,"CO2ppm"]/1000000. # set the melt CO2 content
        melt_wf["H2OT"] = setup.loc[n,"H2O"]/100. # set the H2O melt content
        melt_wf['ST'] = setup.loc[n,"STppm"]/1000000. # set the melt S content
        melt_wf['CT'] = (melt_wf['CO2']/vf.species.loc['CO2','M'])*vf.species.loc['C','M'] # set the melt C content
        melt_wf['HT'] = (melt_wf['H2OT']/vf.species.loc['H2O','M'])*(2.*vf.species.loc['H','M']) # set the melt H content
        melt_wf['Fe3FeT'] = setup.loc[n,"Fe3+FeT"] # set the melt Fe3+/FeT
        fO2 = vf.f_O2(PT,melt_wf,models) # calculate the fO2
        FMQ = vf.fO22Dbuffer(PT,fO2,"FMQ",models) # calculate the fO2 relative to FMQ buffer
        sulfsat = vf.sulfur_saturation(PT,melt_wf,models) # calculate the SCSS, SCAS, StCSS, StCAS, and ST
        sulfide_capacity = vf.C_S(PT,melt_wf) # calculate the sulfide capacity
        sulfate_capacity = vf.C_SO4(PT,melt_wf) # calculate the sulfate capacity
        results1 = pd.DataFrame([[PT['P'],PT["T"],FMQ,sulfsat["SCSS"],sulfsat["StCSS"],sulfsat["SCAS"],sulfsat["StCAS"],sulfsat["ST"],setup.loc[n,"MgO"],sulfide_capacity,sulfate_capacity]]) # collate results
        if n == 0.:
            results_headers = pd.DataFrame([["P","T","FMQ","SCSS","StCSS","SCAS","StCAS",'ST',"MgO","Csulfide","Csulfate"]]) # create result headers
            results = pd.concat([results_headers, results1])
        else:
            results = pd.concat([results, results1])
    results.columns = results.iloc[0]
    results = results[1:]
    results.reset_index(inplace=True)  
    return results

# calculate sulfur-related variables with variable oxygen fugacity.
def calc_sulfur_vfO2(fO2_start,fO2_end,setup,step_size=0.1,models=vf.default_models):
    # fO2_start, fO2_end, and step_size in FMQ
    PT = {"T":setup["T_C"],"P":setup["P_bar"]} # set P and T
    K1 = vf.KHOSg(PT) # calculate equilibrium constant for H2S formation
    K2 = vf.KOSg(PT) # calculate equilibrium constant for SO2 formation
    melt_wf=vf.melt_comp(0.,setup) # set melt composition
    melt_wf['CO2'] = setup.loc[0.,"CO2ppm"]/1000000. # set the melt CO2 content
    melt_wf["H2OT"] = setup.loc[0.,"H2O"]/100. # set the H2O melt content
    melt_wf['CT'] = (melt_wf['CO2']/vf.species.loc['CO2','M'])*vf.species.loc['C','M'] # set the melt C content
    melt_wf['HT'] = (melt_wf['H2OT']/vf.species.loc['H2O','M'])*(2.*vf.species.loc['H','M']) # set the melt H content
    fH2O = vf.f_H2O(PT,melt_wf,models)
    end = int(((abs(fO2_end) + abs(fO2_start))/step_size)+1.) # calculate the end point of the loop
    for n in range(0,end,1):
        fO2_FMQ = fO2_start+(n*step_size) # set FMQ
        fO2 = vf.Dbuffer2fO2(PT,fO2_FMQ,"FMQ",models) # calculate fO2
        melt_wf['Fe3FeT'] = vf.fO22Fe3FeT(fO2,PT,melt_wf,models) # calculate Fe3+/FeT
        SO2_H2S = (K2*fO2**1.5)/(K1*fH2O) # calculate SO2/H2S ratio
        SO2_ST = float(SO2_H2S/(SO2_H2S+1.)) # convert to SO2/ST
        CSO4 = vf.C_SO4(PT,melt_wf,models) # calculate sulfate capacity
        CS = vf.C_S(PT,melt_wf,models) # calculate sulfide capacity
        S6S2 = float((CSO4/CS)*fO2**2.) # calculate S6+/S2- ratio in the melt assuming *S2- and S6+ are the only S-bearing melt species (no H2S)
        S6pST = S6S2/(S6S2+1.) # convert to S6+/ST
        SCSS = vf.SCSS(PT,melt_wf,models) # calculate SCSS
        SCAS = vf.SCAS(PT,melt_wf,models) # calculate SCAS
        STCSS = SCSS/(1.-S6pST) # calculate total S at SCSS
        STCAS = SCAS/S6pST # calculate total S at SCAS
        if STCSS < STCAS: # work out whether STCSS or STCAS is lower
            ST = STCSS
        else:
            ST =  STCAS
        ST_s2 = ST*(1.-S6pST) # calculate S2- content of the melt
        ST_s6 =ST*(S6pST) # calculate S6+ content of the melt
        results1 = pd.DataFrame([[PT['P'],PT["T"],fH2O,fO2_FMQ,SO2_ST,S6pST,SCSS,SCAS,STCSS,STCAS,ST,ST_s2,ST_s6]]) # collate results
        if n == 0.:
            results_headers = pd.DataFrame([["P","T","fH2O","fO2_FMQ","SO2/(SO2+H2S)","S6+/ST","SCSS","SCAS",'STCSS','STCAS','ST','ST_S2-','ST_S6+']]) # create headers
            results = pd.concat([results_headers, results1])
        else:
            results = pd.concat([results, results1])
    results.columns = results.iloc[0]
    results = results[1:]
    results.reset_index(inplace=True)  
    return results


## Melt composition during fractional crystallisation

We use the melt compositions generated during fractionation crystallisation at P = 1000 bar shown in Figure 6 of Wieser & Gleeson (2022) calculated using PyMETScalc (Gleeson et al., 2023).

Wieser, P. and Gleeson, M. (2023) “PySulfSat: An open-source Python3 tool for modeling sulfide and sulfate saturation”, Volcanica, 6(1), pp. 107–127. https//.doi.org/10.30909/vol.06.01.107127

Gleeson, M., A. Paula, and P. Wieser (2023). “PyMELTSCalc”. Zenodo. https://doi.org10.5281/zenodo.7758494. Github Code repository: www.github.com/gleesonm1/pyMELTScalc 

In [3]:
frac_xst = pd.read_csv("files/model_Wieser+22_frac_xst.csv") 

In [4]:
# Extract the initial composition of the fractional crystallisation path
my_analysis = {'Sample':'start',
           'T_C': frac_xst.loc[0,'T_C'], # Temperature in 'C
           'P_bar': frac_xst.loc[0,'P_bar'], # pressure in bar
            'SiO2': frac_xst.loc[0,'SiO2'], # wt%
           'TiO2': frac_xst.loc[0,'TiO2'], # wt%
           'Al2O3': frac_xst.loc[0,'Al2O3'], # wt%
           'FeOT': frac_xst.loc[0,'FeOT'], # wt%
           'MnO': frac_xst.loc[0,'MnO'], # wt%
           'MgO': frac_xst.loc[0,'MgO'], # wt%
           'CaO': frac_xst.loc[0,'CaO'], # wt%
           'Na2O': frac_xst.loc[0,'Na2O'], # wt%
           'K2O': frac_xst.loc[0,'K2O'], # wt%
           'P2O5': frac_xst.loc[0,'P2O5'], # wt%
           'H2O': frac_xst.loc[0,'H2O'], # wt%
           'CO2ppm': frac_xst.loc[0,'CO2ppm']} # ppm

# Turn the dictionary into a pandas dataframe, setting the index to 0.
dry_basalt = pd.DataFrame(my_analysis, index=[0])

In [5]:
# Extract the final composition of the fractional crystallisation path
my_analysis = {'Sample':'final',
           'T_C': frac_xst.loc[50,'T_C'], # Temperature in 'C
           'P_bar': frac_xst.loc[50,'P_bar'], # pressure in bar
            'SiO2': frac_xst.loc[50,'SiO2'], # wt%
           'TiO2': frac_xst.loc[50,'TiO2'], # wt%
           'Al2O3': frac_xst.loc[50,'Al2O3'], # wt%
           'FeOT': frac_xst.loc[50,'FeOT'], # wt%
           'MnO': frac_xst.loc[50,'MnO'], # wt%
           'MgO': frac_xst.loc[50,'MgO'], # wt%
           'CaO': frac_xst.loc[50,'CaO'], # wt%
           'Na2O': frac_xst.loc[50,'Na2O'], # wt%
           'K2O': frac_xst.loc[50,'K2O'], # wt%
           'P2O5': frac_xst.loc[50,'P2O5'], # wt%
           'H2O': frac_xst.loc[50,'H2O'], # wt%
           'CO2ppm': frac_xst.loc[50,'CO2ppm']} # ppm

# Turn the dictionary into a pandas dataframe, setting the index to 0.
wet_andesite = pd.DataFrame(my_analysis, index=[0])

## Calculate sulfur-related variables along the fractional crystallisation path

In [6]:
fx = calc_sulfur_vcomp(frac_xst, models=my_models)

## Calculate sulfur-related variables with varying fO2 for the dry basalt and wet andesite

In [7]:
dry_basalt_fO2 = calc_sulfur_vfO2(-4,+4,dry_basalt,step_size=0.01,models=my_models)
wet_andesite_fO2 = calc_sulfur_vfO2(-4,+4,wet_andesite,step_size=0.01,models=my_models)

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/model_dependent_variables.py:1858: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  K = 10.**((-8117.0/T_K)+0.188*math.log10(T_K)-0.352)
/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/model_dependent_variables.py:3288: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  KD1 = math.exp((-DH/(R*T_K)) + (DS/R) - (DCp/R)*(1.0 - (T0/T_K) - math.log(T_K/T0)) - (1.0/(R*T_K))*D4X - ((DV*(P_Pa-P0))/(R*T_K)) - ((DVdot*(T_K-T0)*(P_Pa-P0))/(R*T_K)) - (DVdash/(2.0*R*T_K))*pow((P_Pa-P0),2.0))
/var/folders/4z/p1y5b5h10yd0hhzf32bhk_440000gn/T/ipykernel_93842/3271021444.py:59: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  SO2_ST = float(SO2_H2S/(SO2_H2S+1.)) # convert to 

## Experimental and natural data for comparison

### fO2 ranges for natural basalts from Cottrell et al. (2021)

Cottrell, E., Birner, S.K., Brounce, M., Davis, F.A., Waters, L.E. and Kelley, K.A., 2021. Oxygen fugacity across tectonic settings. Magma redox geochemistry, pp.33-61. https://doi.org/10.1002/9781119473206.ch3

In [8]:
cottrell21 = pd.read_csv("files/data_Cottrell+21.csv") 

### S6+/ST and fO2 for dry basalt from Saper et al. (2024)

1 atmosphere furnace, Hawaiian basaltic melt inclusions, <0.5 wt% H2O, T = 1225 'C, internal melt inclusion P < 500 bar.

Saper, L.M., Baker, M.B., Brounce, M., Hughes, E.C., Hofmann, A.E. and Stolper, E.M., 2024. Experimental constraints on iron and sulfur redox equilibria and kinetics in basaltic melt inclusions. Geochimica et Cosmochimica Acta. https://doi.org/10.1016/j.gca.2024.07.018

In [9]:
saper24 = pd.read_csv("files/data_Saper+24_GCA.csv")

### S6+/ST and fO2 for wet andesite from Botcharnikov et al. (2011)

Internally heated pressure vessel, synthetic andesite, 3.54-4.74 wt% H2O, T = 1050 'C, P = 2000 bar.

Botcharnikov, R.E., Linnen, R.L., Wilke, M., Holtz, F., Jugo, P.J. and Berndt, J., 2011. High gold concentrations in sulphide-bearing magma under oxidizing conditions. Nature Geoscience, 4(2), pp.112-115. https://doi.org/10.1038/ngeo1042

In [10]:
botcharnikov11 = pd.read_csv("files/data_Botcharnikov+11_NatGeo.csv")

### ST and fO2 for dry basalt at sulfide/anhydrite saturation from Jugo et al. (2005)

Piston-cylinder, synthetic basalt, anhydrous, T = 1300 'C, P = 10000 bar.

Jugo, P.J., Luth, R.W. and Richards, J.P., 2005. An experimental study of the sulfur content in basaltic melts saturated with immiscible sulfide or sulfate liquids at 1300 C and 1· 0 GPa. Journal of Petrology, 46(4), pp.783-798. https://doi.org/10.1093/petrology/egh097

In [11]:
jugo05 = pd.read_csv("files/data_Jugo+05_JPet.csv")

### ST and fO2 for wet andesite at sulfide/anhydrite saturation from Carroll & Rutherford (1985, 1987)

TZM and internally heated pressure vessels, El Chichon andesite, >4 wt%H2O, T = 950-1027 'C, P = 1950-2150 bar.

Carroll, M.R. and Rutherford, M.J., 1987. The stability of igneous anhydrite: experimental results and implications for sulfur behavior in the 1982 El Chichon trachyandesite and other evolved magmas. Journal of Petrology, 28(5), pp.781-801, https://doi.org/10.1093/petrology/28.5.781 

Carroll, M.R. and Rutherford, M.J., 1985. Sulfide and sulfate saturation in hydrous silicate melts. Journal of Geophysical Research: Solid Earth, 90(S02), pp.C601-C612. https://doi.org/10.1029/JB090iS02p0C601 

In [12]:
carroll857 = pd.read_csv("files/data_Carroll8587_subset.csv") 

## Plotting

In [40]:
fig = make_subplots(rows=3, cols=1, shared_yaxes = False, shared_xaxes = False,vertical_spacing=0.07, horizontal_spacing=0.02)

# definte attributes about symbols and curves
lw = 2
syms = 7
lc1 = '#800080'  # purple, dry basalt
lc2 = '#FF8C03' # orange, andesite
lc3 = 'black'
lc4 = 'lightgrey'
lc5 = 'darkgrey'

c = 1
# S6+/ST or SO2/(SO2+H2S) vs. fO2
r = 1
# fO2 ranges from natural basalts from Cottrell et al. (2021)
fig.add_trace(go.Scatter(mode = "markers", x=cottrell21['DQFM'], y=cottrell21['plotting'], 
                         marker=dict(symbol = "circle", size = 10, color=lc4, line_color=lc5, line_width=2), 
                         error_x=dict(type='data', array=cottrell21['sd'],visible=True,width=0)),
                         secondary_y=False, row = r, col = c)
# Experimental data for dry basalts from Saper et al. (2024)
fig.add_trace(go.Scatter(mode = "markers", x=saper24['ΔFMQ'], y=saper24['S6+/∑S'], 
                         marker=dict(symbol = "circle", size = syms, color=lc1, line_color="black", line_width=1), 
                         error_y=dict(type='data', array=saper24['1σ[S6+/∑S]'],visible=True,width=0)),
                         secondary_y=False, row = r, col = c)
# Experimental data for wet andesites from Botcharnikov et al. (2011)
fig.add_trace(go.Scatter(mode = "markers", x=botcharnikov11['DFMQ'], y=botcharnikov11['S6+/ST'], 
                         marker=dict(symbol = "circle", size = syms, color=lc2, line_color="black", line_width=1)),
                         secondary_y=False, row = r, col = c)
# Calculated results for dry basalt
fig.add_trace(go.Scatter(mode = "lines", x=dry_basalt_fO2['fO2_FMQ'], y=dry_basalt_fO2['SO2/(SO2+H2S)'], line_color = lc1, line_width = lw,line_dash = "dash"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=dry_basalt_fO2['fO2_FMQ'], y=dry_basalt_fO2['S6+/ST'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
# Calculated results for wet andesite
fig.add_trace(go.Scatter(mode = "lines", x=wet_andesite_fO2['fO2_FMQ'], y=wet_andesite_fO2['SO2/(SO2+H2S)'], line_color = lc2, line_width = lw,line_dash = "dash"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=wet_andesite_fO2['fO2_FMQ'], y=wet_andesite_fO2['S6+/ST'], line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
# axes
fig.update_xaxes(title = "log<sub>10</sub><i>f</i><sub>O<sub>2</sub></sub> (\u0394FMQ)", range=[-2,3],dtick=1, row = r, col = c)
fig.update_yaxes(title = "S<sup>6+</sup>/S<sub>T</sub> or SO<sub>2</sub>/(SO<sub>2</sub>+H<sub>2</sub>S)",  tickformat = ".1f", range=[0,1], row = r, col = c)


# S content at sulfide and/or anhydrite saturation against fO2
r = 2
# Experimental data for dry basalts from Jugo et al. (2005)
fig.add_trace(go.Scatter(mode = "markers", x=jugo05['DFMQ'], y=jugo05['S'], 
                         marker=dict(symbol = 'circle', size = syms, color=lc1, line_color="black", line_width=1), 
                         error_y=dict(type='data', array=jugo05['S_sd'],visible=True,width=0),
                         error_x=dict(type='data', array=jugo05['DFMQ_sd'],visible=True,width=0)),
                         secondary_y=False, row = r, col = c)
# Experimental data for wet andesites from Carroll & Rutherford (1985, 1987)
fig.add_trace(go.Scatter(mode = "markers", x=carroll857['DFMQ'], y=carroll857['S']/10000., 
                         marker=dict(symbol = 'circle', size = syms, color=lc2, line_color="black", line_width=1), 
                         error_y=dict(type='data', array=carroll857['S_sd']/10000.,visible=True,width=0)),
                         secondary_y=False, row = r, col = c)
# Calculated results for dry basalt
fig.add_trace(go.Scatter(mode = "lines", x=dry_basalt_fO2['fO2_FMQ'], y=dry_basalt_fO2['SCAS']/10000., line_color = lc1, line_width = lw,line_dash = "dash"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=dry_basalt_fO2['fO2_FMQ'], y=dry_basalt_fO2['SCSS']/10000., line_color = lc1, line_width = lw,line_dash = "dot"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=dry_basalt_fO2['fO2_FMQ'], y=dry_basalt_fO2['ST']/10000., line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
# Calculated results for wet andesite
fig.add_trace(go.Scatter(mode = "lines", x=wet_andesite_fO2['fO2_FMQ'], y=wet_andesite_fO2['SCAS']/10000., line_color = lc2, line_width = lw,line_dash = "dash"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=wet_andesite_fO2['fO2_FMQ'], y=wet_andesite_fO2['SCSS']/10000., line_color = lc2, line_width = lw,line_dash = "dot"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=wet_andesite_fO2['fO2_FMQ'], y=wet_andesite_fO2['ST']/10000., line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
# axes
fig.update_yaxes(title = "S (wt%)", range=[0,2], tickformat = ".1f", row = r, col = c)
fig.update_xaxes(title = "log<sub>10</sub><i>f</i><sub>O<sub>2</sub></sub> (\u0394FMQ)", range=[-2,3],dtick=1,row = r, col = c)

# SCSS or SCAS against MgO and T
r=3
fig.add_trace(go.Scatter(mode = "lines", x=fx['MgO'], y=fx['SCSS']/10000., line_color = lc3, line_width = lw,line_dash = "dot"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=fx['MgO'], y=fx['SCAS']/10000., line_color = lc3, line_width = lw,line_dash = "dash"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=[fx.loc[0,'MgO'],fx.loc[0,'MgO']], y=(fx.loc[0,'SCSS']/10000.,fx.loc[0,'SCAS']/10000.), 
                         marker=dict(symbol = "circle", size = syms, color=lc1, line_color="black", line_width=1)), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=[fx.loc[len(fx)-1,'MgO'],fx.loc[len(fx)-1,'MgO']], y=(fx.loc[len(fx)-1,'SCSS']/10000.,fx.loc[len(fx)-1,'SCAS']/10000.), 
                         marker=dict(symbol = "circle", size = syms, color=lc2, line_color="black", line_width=1)), row = r, col = c)
# axes
fig.update_yaxes(title = "S (wt%)", range=[-1.5,0.2], type='log',exponentformat='power', row = r, col = c)
fig.update_xaxes(title = 'MgO (wt%)', range=[0,7],dtick=1,row = r, col = c)

# add antotation
c=1
r=1
fig.add_annotation(x=-1.8, y=0.95,text="(a)",showarrow=False, font=dict(size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=-1.45, y=0.88,text="<i>model</i>",showarrow=False,
                   ax = 42, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=-1, y=0.81,text="\u2014 basalt 1166 \u2103",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=-0.95, y=0.74,text="0.5 wt% H<sub>2</sub>O",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=-1.3, y=0.67,text="\u2014 andesite",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=-1.2, y=0.60,text="1000 \u2103",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=-1., y=0.53,text="1.8 wt% H<sub>2</sub>O",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=-1.45, y=0.46,text="\u2013 \u2013 vapor",showarrow=False,
                   ax = 42, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=-1.47, y=0.39,text="\u2014 melt",showarrow=False,
                   ax = 42, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=1.9, y=0.38,text="<i>data</i>",showarrow=False,
                   ax = 42, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=2.2, y=0.31,text="\u25CF dry basalt",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=2.2, y=0.24,text="1225 \u2103",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=2.3, y=0.17,text="\u25CF wet andesite",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=2.2, y=0.1,text="1025 \u2103",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=0.3, y=0.6,text="MORB",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=0.75, y=0.7,text="BAB",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=0.4, y=0.8,text="arc",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=0.55, y=0.87,text="plume",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)

r=2
fig.add_annotation(x=-1.8, y=1.9,text="(b)",showarrow=False, font=dict(size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=-1.45, y=1.76,text="<i>model</i>",showarrow=False,
                   ax = 42, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=-0.4, y=1.62,text="\u2014 basalt 1166 \u2103 0.5 wt% H<sub>2</sub>O",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=-0.3, y=1.48,text="\u2014 andesite 1000 \u2103 1.8 wt% H<sub>2</sub>O",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=-0.45, y=1.34,text="\u2014 S<sub>T</sub>  \u22C5\u22C5\u22C5 S<sup>2-</sup>CSS  \u2013 \u2013 S<sup>6+</sup>CAS",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
fig.add_annotation(x=-1.55, y=1.00,text="<i>data</i>",showarrow=False,
                   ax = 42, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=-0.8, y=0.86,text="\u25CF dry basalt 1300 \u2103",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=-0.62, y=0.72,text="\u25CF wet andesite ~1000 \u2103",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)

r=3
fig.add_annotation(x=0.3, y=0.1,text="(c)",showarrow=False, font=dict(size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=5, y=-0.67,text="S<sup>2-</sup>CSS",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=5, y=-0.2,text="S<sup>6+</sup>CAS",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
tickh=0.15
fig.add_annotation(x=6.31, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=5.84, y=tickh,text="1150",showarrow=True,
                   ax = 0, ay = -16, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=5.46, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=5.18, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=4.88, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=4.44, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=4.08, y=tickh,text="1100",showarrow=True,
                   ax = 0, ay = -16, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=3.71, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=3.11, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=2.63, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=2.22, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=1.84, y=tickh,text="1050",showarrow=True,
                   ax = 0, ay = -16, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=1.52, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=1.25, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=1.02, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.83, y=tickh,text="",showarrow=True,
                   ax = 0, ay = -9, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.67, y=tickh,text="1000",showarrow=True,
                   ax = 0, ay = -16, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=6.7, y=tickh+0.05,text="<i>T</i> (\u2103)",showarrow=True,arrowcolor="white",
                   ax = 0, ay = -10, font=dict(color="black"), textangle=0, row = r, col = c)

fig.update_layout(height=1000, width=450, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black", tickfont=dict(size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black", tickfont=dict(size=12))

fig.update_layout(font_family="Arial",font_color="black")

fig.show()